In [0]:
import tensorflow as tf

In [0]:
class CausalConv1D(tf.keras.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=None,
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
        
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        if self.data_format == 'channels_first':
            inputs = tf.pad(inputs, tf.constant([[0, 0], [0, 0], [padding, 0]], dtype=tf.int32))
        else:
            inputs = tf.pad(inputs, tf.constant([(0, 0,), (padding, 0), (0, 0)]))
        return super(CausalConv1D, self).call(inputs)
      
      
class TemporalBlock(tf.keras.layers.Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout=0.2, 
                 trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalBlock, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )        
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv1")
        self.conv2 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv2")
        self.down_sample = None

    
    def build(self, input_shape):
        channel_dim = 2
        self.dropout1 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        if input_shape[channel_dim] != self.n_outputs:
            self.down_sample = tf.layers.Dense(self.n_outputs, activation=None)
        self.built = True
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        if self.down_sample is not None:
            inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)
      
    def compute_output_shape(self, input_shape):
      shape = tf.TensorShape(input_shape).as_list()
      shape[-1] = self.n_outputs
      return tf.TensorShape(shape)
      

In [4]:
x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
"""
with tf.variable_scope("tcn"):
    container = tf.keras.models.Sequential()
    conv = CausalConv1D(8, 3, activation=tf.nn.relu)
    container.add(conv)
# output = conv(x)
output = container(x)
print(conv.count_params())
container.summary()
"""
"""
conv = CausalConv1D(8, 3, activation=tf.nn.relu)
container = tf.keras.models.Sequential()
container.add(conv)
output = container(x, training=tf.constant(True))
container.summary()
"""
tblock = TemporalBlock(8, 2, 1, 1)
container = tf.keras.models.Sequential()
container.add(tblock)
output = container(x, training=tf.constant(True))
container.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
temporal_block (TemporalBloc multiple                  0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [0]:
class DenseNetwork(tf.keras.Model):

  def __init__(self, hidden_size, num_layers=1, **kwargs):
    super(DenseNetwork, self).__init__(**kwargs)
    
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.layer_list = []
    for i in range(self.num_layers):
      self.layer_list.append(tf.keras.layers.Dense(self.hidden_size))

  def call(self, inputs):
    current_inp = inputs
    for layer in self.layer_list:
      current_inp = layer(current_inp)
    return current_inp
  
  def get_config(self):
    base_config = super(DenseNetwork, self).get_config()
    return base_config
  

class DenseLayerSeq(tf.keras.models.Sequential):

  def __init__(self, hidden_size, num_layers=1, **kwargs):
    super(DenseLayerSeq, self).__init__(**kwargs)
    
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    for i in range(self.num_layers):
      self.add(tf.keras.layers.Dense(self.hidden_size, name=self.name + "_" + str(i)))

  def call(self, inputs, training=None, mask=None):
    out = super(DenseLayerSeq, self).call(inputs, training=training, mask=mask)
    return out
  
  def get_config(self):
    base_config = super(DenseLayerSeq, self).get_config()
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)
  

class AutoEncoder(tf.keras.Model):

  def __init__(self, num_encoder_layers, num_decoder_layers, hidden_size, latent_size, **kwargs):
    super(AutoEncoder, self).__init__(**kwargs)
    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers = num_decoder_layers
    self.hidden_size = hidden_size
    self.latent_size = latent_size
    
    # self.encoder = DenseNetwork(hidden_size, num_encoder_layers, name="encoder")
    self.encoder = DenseLayerSeq(hidden_size, num_encoder_layers)
    self.embedding = tf.keras.layers.Dense(latent_size, name="embedding")
    # self.decoder = DenseNetwork(hidden_size, num_decoder_layers, name="decoder")
    self.decoder = DenseLayerSeq(hidden_size, num_decoder_layers, name="decoder")
    self.reconstruction = None
    
  def build(self, input_shape):
    self.reconstruction = tf.keras.layers.Dense(input_shape[1], name="reconstruction")

  def call(self, inputs):
    encoded = self.embedding(self.encoder(inputs))
    decoded = self.reconstruction(self.decoder(encoded))
    return decoded
  
  def get_config(self):
    base_config = super(DenseLayerSeq, self).get_config()
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [18]:
# model = tf.keras.Sequential([MyModel(10), tf.keras.layers.Activation('softmax')])
# model1 = MyModel(20, 2)
# model2 = MyModel(20)

x = tf.random_normal((32, 10)) # (batch_size, length, channel)
# preds = model1(x)
# preds2 = model2(preds)
# model1.summary()
# model2.summary()

autoencoder = AutoEncoder(3, 3, 20, 4)
reconstruction = autoencoder(x)
autoencoder.summary()

dense_layer_seq
decoder
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_seq (DenseLayerS multiple                  1060      
_________________________________________________________________
embedding (Dense)            multiple                  84        
_________________________________________________________________
decoder (DenseLayerSeq)      multiple                  940       
_________________________________________________________________
reconstruction (Dense)       multiple                  210       
Total params: 2,294
Trainable params: 2,294
Non-trainable params: 0
_________________________________________________________________


In [19]:
autoencoder.encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_seq_0 (Dense)    multiple                  220       
_________________________________________________________________
dense_layer_seq_1 (Dense)    multiple                  420       
_________________________________________________________________
dense_layer_seq_2 (Dense)    multiple                  420       
Total params: 1,060
Trainable params: 1,060
Non-trainable params: 0
_________________________________________________________________


In [0]:
encoder = DenseLayerSeq(20, 2, name="encoder")
encoder(x)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              multiple                  220       
_________________________________________________________________
dense_8 (Dense)              multiple                  420       
Total params: 640
Trainable params: 640
Non-trainable params: 0
_________________________________________________________________
